# Betfair Real-Time Market Monitor

Advanced real-time monitoring dashboard for Betfair trading data with automated alerts and comprehensive market analysis.

## Features
- **Real-time Data Streaming**: Continuous market data updates
- **Price Alert System**: Automated notifications for price movements
- **Live Charts**: Dynamic updating charts with professional styling
- **Market Dashboard**: Comprehensive trading statistics
- **Volume Analysis**: Advanced volume pattern detection
- **Trend Detection**: Automated trend analysis and signals

In [ ]:
// Enhanced package setup for real-time monitoring
#r "nuget: FSharp.Data"
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: Newtonsoft.Json"
#r "nuget: System.Reactive"

open System
open System.Net.Http
open System.Threading
open System.Threading.Tasks
open FSharp.Data
open Plotly.NET
open Plotly.NET.LayoutObjects
open Newtonsoft.Json
open Newtonsoft.Json.Linq
open System.Reactive.Linq
open System.Reactive.Subjects

// Enable Plotly.NET interactive display
Plotly.NET.Defaults.DefaultDisplayOptions <- DisplayOptions.init(PlotlyJSReference = PlotlyJSReference.Require)

In [ ]:
// Enhanced data models for real-time monitoring
type MarketSnapshot = {
    Timestamp: DateTime
    MarketId: string
    EventName: string
    MarketName: string
    StartTime: DateTime
    Status: string
}

type SelectionSnapshot = {
    SelectionId: string
    Name: string
    Price: decimal
    Volume: decimal
    LastUpdate: DateTime
}

type TradingEntry = {
    Time: DateTime
    Price: decimal
    Volume: decimal
}

type MarketData = {
    Market: MarketSnapshot
    Selection: SelectionSnapshot
    TradingHistory: TradingEntry list
    LastUpdated: DateTime
}

type PriceAlert = {
    AlertId: string
    SelectionName: string
    AlertType: string // "PriceIncrease", "PriceDecrease", "VolumeSpike"
    Threshold: decimal
    CurrentValue: decimal
    Timestamp: DateTime
    Message: string
}

type MarketStatistics = {
    TotalVolume: decimal
    PriceRange: decimal * decimal
    PriceChange: decimal
    PriceChangePercent: decimal
    VolatilityScore: decimal
    TrendDirection: string // "Up", "Down", "Sideways"
    Confidence: decimal
}

In [ ]:
// Real-time monitoring configuration
let mutable monitoringActive = false
let mutable refreshInterval = 5000 // 5 seconds
let httpClient = new HttpClient()

// Market configuration - can be changed for different markets
let mutable currentMarketId = "1.244137767"
let mutable currentSelectionId = "500893_0.00"
let baseUrl = "http://localhost:10043"
let dataContextName = "MarketSelectionsPriceHistoryData"

// Alert thresholds
let mutable priceChangeThreshold = 0.05m // 5% price change
let mutable volumeSpikeThreshold = 2.0m // 2x average volume

// Data storage for real-time updates
let mutable lastMarketData: MarketData option = None
let mutable marketHistory: MarketData list = []
let mutable priceAlerts: PriceAlert list = []

printfn "🔧 Real-time monitoring configured"
printfn "📊 Market ID: %s" currentMarketId
printfn "🎯 Selection ID: %s" currentSelectionId
printfn "⏱️ Refresh interval: %d ms" refreshInterval

In [ ]:
// Enhanced API fetching with comprehensive error handling
let buildApiUrl marketId selectionId =
    sprintf "%s/api/getDataContextForBetfairMarketSelection?dataContextName=%s&marketId=%s&selectionId=%s"
        baseUrl dataContextName marketId selectionId

let fetchMarketDataAsync marketId selectionId = async {
    try
        let url = buildApiUrl marketId selectionId
        let! response = httpClient.GetStringAsync(url) |> Async.AwaitTask
        
        let json = JObject.Parse(response)
        let isError = json.["isError"]?.Value<bool>() |> Option.defaultValue false
        
        if isError then
            let message = json.["message"]?.Value<string>() |> Option.defaultValue "Unknown API error"
            return Error (sprintf "API Error: %s" message)
        else
            // Parse market data
            let marketJson = json.["market"]
            let market = {
                Timestamp = DateTime.Now
                MarketId = marketId
                EventName = marketJson.["eventName"].Value<string>()
                MarketName = marketJson.["marketName"].Value<string>()
                StartTime = marketJson.["startTime"].Value<DateTime>()
                Status = "Active" // Could be enhanced with actual status from API
            }
            
            // Parse selection data
            let selectionsJson = json.["selections"] :?> JArray
            let selectionJson = selectionsJson.[0]
            let selection = {
                SelectionId = selectionId
                Name = selectionJson.["selection"].["name"].Value<string>()
                Price = selectionJson.["selection"].["price"].Value<decimal>()
                Volume = 0m // Will be calculated from trading history
                LastUpdate = DateTime.Now
            }
            
            // Parse trading history
            let tradingJson = selectionJson.["tradedPricesAndVolume"] :?> JArray
            let tradingHistory = 
                tradingJson
                |> Seq.map (fun item -> {
                    Time = item.["time"].Value<DateTime>()
                    Price = item.["price"].Value<decimal>()
                    Volume = item.["volume"].Value<decimal>()
                })
                |> List.ofSeq
                |> List.sortBy (_.Time)
            
            // Calculate total volume
            let totalVolume = tradingHistory |> List.sumBy (_.Volume)
            let updatedSelection = { selection with Volume = totalVolume }
            
            let marketData = {
                Market = market
                Selection = updatedSelection
                TradingHistory = tradingHistory
                LastUpdated = DateTime.Now
            }
            
            return Ok marketData
            
    with
    | ex -> return Error (sprintf "Network/Parse Error: %s" ex.Message)
}

// Test initial connection
let testConnection () = async {
    printfn "🔍 Testing API connection..."
    let! result = fetchMarketDataAsync currentMarketId currentSelectionId
    match result with
    | Ok data ->
        printfn "✅ API connection successful!"
        printfn "📊 Market: %s" data.Market.EventName
        printfn "🎯 Selection: %s" data.Selection.Name
        printfn "💰 Current Price: £%.2f" data.Selection.Price
        printfn "📈 Trading Records: %d" data.TradingHistory.Length
        return true
    | Error msg ->
        printfn "❌ API connection failed: %s" msg
        return false
}

testConnection () |> Async.RunSynchronously |> ignore

In [ ]:
// Real-time market statistics calculation
let calculateMarketStatistics (currentData: MarketData) (previousData: MarketData option) =
    let tradingHistory = currentData.TradingHistory
    
    if tradingHistory.Length = 0 then
        {
            TotalVolume = 0m
            PriceRange = (0m, 0m)
            PriceChange = 0m
            PriceChangePercent = 0m
            VolatilityScore = 0m
            TrendDirection = "Unknown"
            Confidence = 0m
        }
    else
        let prices = tradingHistory |> List.map (_.Price)
        let minPrice = List.min prices
        let maxPrice = List.max prices
        let currentPrice = currentData.Selection.Price
        let totalVolume = tradingHistory |> List.sumBy (_.Volume)
        
        // Calculate price change if we have previous data
        let (priceChange, priceChangePercent) = 
            match previousData with
            | Some prev when prev.Selection.Price > 0m ->
                let change = currentPrice - prev.Selection.Price
                let changePercent = (change / prev.Selection.Price) * 100m
                (change, changePercent)
            | _ -> (0m, 0m)
        
        // Calculate volatility (standard deviation of prices)
        let avgPrice = prices |> List.average
        let priceVariances = prices |> List.map (fun p -> (p - avgPrice) * (p - avgPrice))
        let volatility = if priceVariances.Length > 1 then sqrt(List.average priceVariances |> float) |> decimal else 0m
        
        // Determine trend direction
        let (trendDirection, confidence) = 
            if tradingHistory.Length >= 5 then
                let recentPrices = tradingHistory |> List.rev |> List.take 5 |> List.rev |> List.map (_.Price)
                let firstPrice = recentPrices |> List.head
                let lastPrice = recentPrices |> List.last
                
                if lastPrice > firstPrice * 1.02m then ("Up", 0.8m)
                elif lastPrice < firstPrice * 0.98m then ("Down", 0.8m)
                else ("Sideways", 0.6m)
            else
                ("Unknown", 0m)
        
        {
            TotalVolume = totalVolume
            PriceRange = (minPrice, maxPrice)
            PriceChange = priceChange
            PriceChangePercent = priceChangePercent
            VolatilityScore = volatility
            TrendDirection = trendDirection
            Confidence = confidence
        }

// Function to generate price alerts
let checkPriceAlerts (currentData: MarketData) (previousData: MarketData option) =
    let stats = calculateMarketStatistics currentData previousData
    let mutable alerts = []
    
    // Price change alerts
    if abs stats.PriceChangePercent > priceChangeThreshold * 100m then
        let alertType = if stats.PriceChangePercent > 0m then "PriceIncrease" else "PriceDecrease"
        let alert = {
            AlertId = Guid.NewGuid().ToString("N").[..7]
            SelectionName = currentData.Selection.Name
            AlertType = alertType
            Threshold = priceChangeThreshold * 100m
            CurrentValue = abs stats.PriceChangePercent
            Timestamp = DateTime.Now
            Message = sprintf "%s: %.2f%% price %s (threshold: %.1f%%)" 
                currentData.Selection.Name 
                (abs stats.PriceChangePercent) 
                (if stats.PriceChangePercent > 0m then "increase" else "decrease")
                (priceChangeThreshold * 100m)
        }
        alerts <- alert :: alerts
    
    // Volume spike alerts
    match previousData with
    | Some prev when prev.TradingHistory.Length > 0 ->
        let previousVolume = prev.TradingHistory |> List.sumBy (_.Volume)
        let currentVolume = currentData.TradingHistory |> List.sumBy (_.Volume)
        let volumeIncrease = if previousVolume > 0m then currentVolume / previousVolume else 1m
        
        if volumeIncrease > volumeSpikeThreshold then
            let alert = {
                AlertId = Guid.NewGuid().ToString("N").[..7]
                SelectionName = currentData.Selection.Name
                AlertType = "VolumeSpike"
                Threshold = volumeSpikeThreshold
                CurrentValue = volumeIncrease
                Timestamp = DateTime.Now
                Message = sprintf "%s: %.1fx volume spike detected" currentData.Selection.Name volumeIncrease
            }
            alerts <- alert :: alerts
    | _ -> ()
    
    alerts

In [ ]:
// Real-time chart creation with live updates
let createLiveMarketChart (marketData: MarketData) (stats: MarketStatistics) =
    let tradingHistory = marketData.TradingHistory
    
    if tradingHistory.Length = 0 then
        Chart.Point([0], [0]) |> Chart.withTitle("No trading data available")
    else
        // Aggregate data by time to match HTML functionality
        let aggregatedData = 
            tradingHistory
            |> List.groupBy (fun entry -> entry.Time.ToString("yyyy-MM-dd HH:mm:ss"))
            |> List.map (fun (timeKey, entries) ->
                let totalVolume = entries |> List.sumBy (_.Volume)
                let latestPrice = entries |> List.last |> (_.Price)
                let timestamp = entries |> List.head |> (_.Time)
                (timestamp, latestPrice, totalVolume))
            |> List.sortBy (fun (time, _, _) -> time)
        
        let times = aggregatedData |> List.map (fun (time, _, _) -> time)
        let prices = aggregatedData |> List.map (fun (_, price, _) -> float price)
        let volumes = aggregatedData |> List.map (fun (_, _, volume) -> float volume)
        
        // Create price trace with enhanced styling
        let priceTrace = 
            Chart.Spline(
                x = times,
                y = prices,
                Name = sprintf "Price (£%.2f)" marketData.Selection.Price,
                LineColor = Color.fromHex "#2E86AB",
                LineWidth = 4.0,
                ShowMarkers = true
            )
            |> Chart.withTraceInfo(YAxis = StyleParam.SubPlotId.YAxis 1)
            |> Chart.withMarkerStyle(
                Size = 10,
                Color = Color.fromHex "#2E86AB",
                Line = Line.init(Color = Color.fromKeyword White, Width = 3.0)
            )
        
        // Create volume trace with gradient colors
        let volumeTrace =
            Chart.Column(
                keys = times,
                values = volumes,
                Name = sprintf "Volume (£%.0f)" stats.TotalVolume,
                MarkerColor = Color.fromHex "#FF9800"
            )
            |> Chart.withTraceInfo(YAxis = StyleParam.SubPlotId.YAxis 2)
            |> Chart.withMarkerStyle(
                Opacity = 0.75,
                Line = Line.init(Color = Color.fromHex "#FF6B35", Width = 1.5)
            )
        
        // Determine title color based on trend
        let titleColor = 
            match stats.TrendDirection with
            | "Up" -> "#27AE60"
            | "Down" -> "#E74C3C"
            | _ -> "#2C3E50"
        
        let trendEmoji = 
            match stats.TrendDirection with
            | "Up" -> "📈"
            | "Down" -> "📉"
            | _ -> "📊"
        
        // Combine charts with professional styling
        [priceTrace; volumeTrace]
        |> Chart.combine
        |> Chart.withTitle(
            sprintf "%s %s - %s (%.2f%% %s)" 
                trendEmoji
                marketData.Market.EventName 
                marketData.Selection.Name 
                (abs stats.PriceChangePercent)
                (if stats.PriceChangePercent >= 0m then "↑" else "↓"),
            TitleFont = Font.init(Size = 24, Color = Color.fromHex titleColor, Family = StyleParam.FontFamily.Arial)
        )
        |> Chart.withXAxisStyle(
            Title = Title.init(sprintf "Time (Last Update: %s)" (DateTime.Now.ToString("HH:mm:ss")), 
                              Font = Font.init(Size = 12, Color = Color.fromHex "#34495e")),
            ShowGrid = true,
            GridColor = Color.fromRGBA(0, 0, 0, 0.1)
        )
        |> Chart.withYAxisStyle(
            Title = Title.init("Price (£)", Font = Font.init(Size = 14, Color = Color.fromHex "#2E86AB")),
            Side = StyleParam.Side.Left,
            ShowGrid = true,
            GridColor = Color.fromRGBA(46, 134, 171, 0.2)
        )
        |> Chart.withYAxis(
            LinearAxis.init(
                Title = Title.init("Volume (£)", Font = Font.init(Size = 14, Color = Color.fromHex "#FF6B35")),
                Side = StyleParam.Side.Right,
                Overlaying = StyleParam.LinearAxisId.Y 1,
                ShowGrid = false
            ),
            Id = StyleParam.SubPlotId.YAxis 2
        )
        |> Chart.withLayoutStyle(
            PaperBGColor = Color.fromHex "#FAFAFA",
            PlotBGColor = Color.fromKeyword White,
            ShowLegend = true,
            Legend = Legend.init(
                X = 0.02,
                Y = 0.98,
                BGColor = Color.fromRGBA(255, 255, 255, 0.95),
                BorderColor = Color.fromHex "#BDC3C7",
                BorderWidth = 1.0
            )
        )
        |> Chart.withConfig(
            Config.init(
                Responsive = true,
                DisplayModeBar = true,
                DisplayLogo = false
            )
        )
        |> Chart.withSize(1200, 700)

In [ ]:
// Real-time monitoring dashboard display
let displayMarketDashboard (marketData: MarketData) (stats: MarketStatistics) (alerts: PriceAlert list) =
    let currentTime = DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss")
    
    printfn "\n🔴 LIVE MARKET DASHBOARD - %s" currentTime
    printfn "═══════════════════════════════════════════════════════════════"
    
    // Market Information
    printfn "\n📊 MARKET INFORMATION"
    printfn "─────────────────────────────────────────────"
    printfn "🏇 Event: %s" marketData.Market.EventName
    printfn "🎯 Market: %s" marketData.Market.MarketName
    printfn "⭐ Selection: %s" marketData.Selection.Name
    printfn "🆔 Market ID: %s" marketData.Market.MarketId
    printfn "🕐 Start Time: %s" (marketData.Market.StartTime.ToString("yyyy-MM-dd HH:mm:ss"))
    printfn "📡 Last Update: %s" (marketData.LastUpdated.ToString("yyyy-MM-dd HH:mm:ss"))
    
    // Current Price & Statistics
    printfn "\n💰 LIVE TRADING DATA"
    printfn "─────────────────────────────────────────────"
    printfn "💵 Current Price: £%.2f" marketData.Selection.Price
    
    let changeColor = if stats.PriceChangePercent >= 0m then "📈" else "📉"
    printfn "%s Price Change: £%.3f (%.2f%%)" changeColor stats.PriceChange stats.PriceChangePercent
    
    let (minPrice, maxPrice) = stats.PriceRange
    printfn "📊 Price Range: £%.2f - £%.2f" minPrice maxPrice
    printfn "💎 Total Volume: £%.2f" stats.TotalVolume
    printfn "⚡ Volatility Score: %.3f" stats.VolatilityScore
    
    // Trend Analysis
    printfn "\n🔍 TREND ANALYSIS"
    printfn "─────────────────────────────────────────────"
    let trendIcon = match stats.TrendDirection with
                    | "Up" -> "🟢"
                    | "Down" -> "🔴"
                    | "Sideways" -> "🟡"
                    | _ -> "⚪"
    printfn "%s Trend Direction: %s" trendIcon stats.TrendDirection
    printfn "🎯 Confidence Level: %.1f%%" (stats.Confidence * 100m)
    printfn "📈 Data Points: %d" marketData.TradingHistory.Length
    
    // Recent Alerts
    printfn "\n🚨 ACTIVE ALERTS"
    printfn "─────────────────────────────────────────────"
    if alerts.Length = 0 then
        printfn "✅ No alerts - market within normal parameters"
    else
        alerts
        |> List.rev  // Show most recent first
        |> List.take (min 5 alerts.Length)  // Show last 5 alerts
        |> List.iteri (fun i alert ->
            let alertIcon = match alert.AlertType with
                          | "PriceIncrease" -> "🟢"
                          | "PriceDecrease" -> "🔴"
                          | "VolumeSpike" -> "🔥"
                          | _ -> "⚠️"
            printfn "%s [%s] %s" alertIcon alert.AlertId alert.Message
        )
    
    printfn "\n═══════════════════════════════════════════════════════════════\n"

// Function to refresh and display live data
let refreshLiveData () = async {
    let! result = fetchMarketDataAsync currentMarketId currentSelectionId
    
    match result with
    | Ok newData ->
        // Store previous data for comparison
        let previousData = lastMarketData
        lastMarketData <- Some newData
        marketHistory <- newData :: (marketHistory |> List.take (min 100 marketHistory.Length))
        
        // Calculate statistics and check alerts
        let stats = calculateMarketStatistics newData previousData
        let newAlerts = checkPriceAlerts newData previousData
        priceAlerts <- (newAlerts @ priceAlerts) |> List.take 20  // Keep last 20 alerts
        
        // Display dashboard
        displayMarketDashboard newData stats newAlerts
        
        // Create and display live chart
        let liveChart = createLiveMarketChart newData stats
        
        return Some (newData, stats, liveChart)
        
    | Error msg ->
        printfn "❌ Failed to refresh data: %s" msg
        return None
}

// Initial data load
printfn "🚀 Loading initial market data..."
let initialLoad = refreshLiveData () |> Async.RunSynchronously

match initialLoad with
| Some (data, stats, chart) ->
    printfn "✅ Initial data loaded successfully!"
    chart  // Display the initial chart
| None ->
    printfn "❌ Failed to load initial data"
    Chart.Point([0], [0]) |> Chart.withTitle("Failed to load market data")

In [ ]:
// Interactive monitoring controls
let startRealTimeMonitoring () =
    if monitoringActive then
        printfn "⚠️ Monitoring already active"
    else
        monitoringActive <- true
        printfn "🟢 Starting real-time monitoring..."
        printfn "📊 Market: %s" currentMarketId
        printfn "🎯 Selection: %s" currentSelectionId
        printfn "⏱️ Refresh interval: %d seconds" (refreshInterval / 1000)
        printfn "🚨 Price alert threshold: %.1f%%" (priceChangeThreshold * 100m)
        printfn "🔥 Volume spike threshold: %.1fx" volumeSpikeThreshold
        printfn "\n🔄 Monitoring will continue until stopRealTimeMonitoring() is called"
        printfn "📈 Call refreshLiveData() |> Async.RunSynchronously to manually update"
        
let stopRealTimeMonitoring () =
    if not monitoringActive then
        printfn "⚠️ Monitoring not currently active"
    else
        monitoringActive <- false
        printfn "🔴 Real-time monitoring stopped"
        
let changeMarket newMarketId newSelectionId =
    currentMarketId <- newMarketId
    currentSelectionId <- newSelectionId
    lastMarketData <- None
    marketHistory <- []
    priceAlerts <- []
    printfn "🔄 Market changed to: %s / %s" newMarketId newSelectionId
    printfn "📊 Data history cleared - ready for new market monitoring"
    
let setAlertThresholds newPriceThreshold newVolumeThreshold =
    priceChangeThreshold <- newPriceThreshold
    volumeSpikeThreshold <- newVolumeThreshold
    printfn "🚨 Alert thresholds updated:"
    printfn "📈 Price change: %.1f%%" (priceChangeThreshold * 100m)
    printfn "🔥 Volume spike: %.1fx" volumeSpikeThreshold
    
let showAlertHistory () =
    printfn "\n🚨 ALERT HISTORY (Last 10)"
    printfn "═══════════════════════════════════════"
    if priceAlerts.Length = 0 then
        printfn "✅ No alerts recorded"
    else
        priceAlerts
        |> List.rev
        |> List.take (min 10 priceAlerts.Length)
        |> List.iteri (fun i alert ->
            let alertIcon = match alert.AlertType with
                          | "PriceIncrease" -> "🟢"
                          | "PriceDecrease" -> "🔴"
                          | "VolumeSpike" -> "🔥"
                          | _ -> "⚠️"
            printfn "%d. %s [%s] %s" 
                (i + 1) 
                (alert.Timestamp.ToString("HH:mm:ss")) 
                alertIcon 
                alert.Message
        )
    printfn ""

// Display monitoring controls
printfn "\n🎛️ MONITORING CONTROLS"
printfn "═══════════════════════════════════════"
printfn "🟢 startRealTimeMonitoring() - Begin monitoring"
printfn "🔴 stopRealTimeMonitoring() - Stop monitoring"
printfn "🔄 refreshLiveData() |> Async.RunSynchronously - Manual refresh"
printfn "📊 changeMarket \"marketId\" \"selectionId\" - Switch markets"
printfn "🚨 setAlertThresholds 0.05m 2.0m - Set price/volume thresholds"
printfn "📜 showAlertHistory() - View recent alerts"
printfn ""
printfn "🚀 Ready for real-time monitoring! Call startRealTimeMonitoring() to begin."

## Quick Start Guide

### 1. Basic Monitoring
```fsharp
// Start monitoring the current market
startRealTimeMonitoring()

// Manually refresh data and view chart
let result = refreshLiveData() |> Async.RunSynchronously

// Stop monitoring
stopRealTimeMonitoring()
```

### 2. Switch Markets
```fsharp
// Change to a different market/selection
changeMarket "1.244183019" "47972"
```

### 3. Customize Alerts
```fsharp
// Set 2% price change and 3x volume spike alerts
setAlertThresholds 0.02m 3.0m
```

### 4. View Alert History
```fsharp
showAlertHistory()
```

In [ ]:
// 🔴 LIVE DEMO - Click to refresh market data
printfn "\n⚡ LIVE MARKET DATA REFRESH"
printfn "═══════════════════════════════════════════"
printfn "Click the button below to refresh live market data:"

// Execute live refresh
let liveRefreshResult = refreshLiveData() |> Async.RunSynchronously

match liveRefreshResult with
| Some (data, stats, chart) ->
    printfn "\n✅ Live data refreshed successfully!"
    printfn "📊 Displaying live chart..."
    
    // Show the live chart
    chart
    
| None ->
    printfn "\n❌ Failed to refresh live data"
    Chart.Point([0], [0]) |> Chart.withTitle("Live data refresh failed")

---

## 🔴 Live Monitoring Session

The monitoring system is now ready. Use the functions above to:

- **Monitor in real-time**: Call `startRealTimeMonitoring()` 
- **Get instant updates**: Run `refreshLiveData() |> Async.RunSynchronously`
- **View alerts**: Use `showAlertHistory()`
- **Change markets**: Call `changeMarket "marketId" "selectionId"`

> **Note**: This notebook replicates the functionality of your HTML file with additional real-time monitoring capabilities, professional chart styling, and automated alert systems.

# Real-Time Betfair Market Monitor

This notebook provides real-time monitoring of Betfair markets with automatic data refresh and live charting capabilities.

## Features:
- Real-time data fetching with configurable intervals
- Live updating charts
- Market alerts and notifications
- Price movement tracking

In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: Newtonsoft.Json"

open System
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data
open Plotly.NET
open Plotly.NET.LayoutObjects
open Newtonsoft.Json.Linq

Plotly.NET.Defaults.DefaultDisplayOptions <- DisplayOptions.init(PlotlyJSReference = PlotlyJSReference.Require)

## Market Configuration

Configure the market and selection you want to monitor.

In [ ]:
// Market monitoring configuration
type MarketConfig = {
    BaseUrl: string
    MarketId: string
    SelectionId: string
    RefreshIntervalSeconds: int
}

let config = {
    BaseUrl = "http://localhost:10043"
    MarketId = "1.244137767"
    SelectionId = "500893_0.00"
    RefreshIntervalSeconds = 30
}

printfn "🔧 Market Monitor Configuration:"
printfn "   📡 API URL: %s" config.BaseUrl
printfn "   🎯 Market ID: %s" config.MarketId
printfn "   ⭐ Selection ID: %s" config.SelectionId
printfn "   ⏱️  Refresh Interval: %d seconds" config.RefreshIntervalSeconds

## Real-Time Data Types

In [ ]:
type MarketSnapshot = {
    Timestamp: DateTime
    EventName: string
    MarketName: string
    SelectionName: string
    CurrentPrice: decimal
    TotalVolume: decimal
    DataPoints: int
}

type PriceAlert = {
    Timestamp: DateTime
    Message: string
    AlertType: string
    Price: decimal
}

// Mutable state for real-time monitoring
let mutable marketHistory: MarketSnapshot list = []
let mutable priceAlerts: PriceAlert list = []

## Real-Time Data Fetcher

In [ ]:
let httpClient = new HttpClient()

let fetchMarketSnapshot (config: MarketConfig) = async {
    try
        let url = $"{config.BaseUrl}/api/getDataContextForBetfairMarketSelection?dataContextName=MarketSelectionsPriceHistoryData&marketId={config.MarketId}&selectionId={config.SelectionId}"
        
        let! response = httpClient.GetStringAsync(url) |> Async.AwaitTask
        let json = JObject.Parse(response)
        
        let isError = json.["isError"]?.Value<bool>() |> Option.defaultValue false
        if isError then
            let message = json.["message"]?.Value<string>() |> Option.defaultValue "Unknown error"
            return Error $"API Error: {message}"
        else
            let marketJson = json.["market"]
            let selectionsJson = json.["selections"] :?> JArray
            let selectionJson = selectionsJson.[0]
            let tradingJson = selectionJson.["tradedPricesAndVolume"] :?> JArray
            
            let totalVolume = 
                tradingJson
                |> Seq.sumBy (fun item -> item.["volume"].Value<decimal>())
            
            let snapshot = {
                Timestamp = DateTime.Now
                EventName = marketJson.["eventName"].Value<string>()
                MarketName = marketJson.["marketName"].Value<string>()
                SelectionName = selectionJson.["selection"].["name"].Value<string>()
                CurrentPrice = selectionJson.["selection"].["price"].Value<decimal>()
                TotalVolume = totalVolume
                DataPoints = tradingJson.Count
            }
            
            return Ok snapshot
    with
    | ex -> return Error $"Failed to fetch data: {ex.Message}"
}

## Price Alert System

In [ ]:
// Price alert configuration
let priceChangeThreshold = 0.05m // 5% price change
let volumeThreshold = 1000m      // £1000 volume threshold

let checkForAlerts (current: MarketSnapshot) (previous: MarketSnapshot option) =
    let alerts = ResizeArray<PriceAlert>()
    
    // Check volume threshold
    if current.TotalVolume > volumeThreshold then
        alerts.Add({
            Timestamp = current.Timestamp
            Message = $"High volume detected: £{current.TotalVolume:F2}"
            AlertType = "VOLUME"
            Price = current.CurrentPrice
        })
    
    // Check price changes
    match previous with
    | Some prev when prev.CurrentPrice > 0m ->
        let priceChangePercent = abs (current.CurrentPrice - prev.CurrentPrice) / prev.CurrentPrice
        if priceChangePercent > priceChangeThreshold then
            let direction = if current.CurrentPrice > prev.CurrentPrice then "UP" else "DOWN"
            alerts.Add({
                Timestamp = current.Timestamp
                Message = $"Significant price movement {direction}: {priceChangePercent * 100m:F1}%"
                AlertType = "PRICE_CHANGE"
                Price = current.CurrentPrice
            })
    | _ -> ()
    
    alerts |> List.ofSeq

let displayAlerts (alerts: PriceAlert list) =
    if not alerts.IsEmpty then
        printfn "🚨 ALERTS:"
        alerts
        |> List.iter (fun alert ->
            let icon = match alert.AlertType with
                       | "VOLUME" -> "💰"
                       | "PRICE_CHANGE" -> "📈"
                       | _ -> "⚠️"
            printfn "   %s %s [£%.2f] at %s" 
                icon alert.Message (float alert.Price) (alert.Timestamp.ToString("HH:mm:ss"))
        )

## Live Market Dashboard

In [ ]:
let displayMarketDashboard (snapshot: MarketSnapshot) =
    printfn "\n📊 LIVE MARKET DASHBOARD"
    printfn "========================="
    printfn "🕐 Last Update: %s" (snapshot.Timestamp.ToString("yyyy-MM-dd HH:mm:ss"))
    printfn "🏇 Event: %s" snapshot.EventName
    printfn "🎯 Market: %s" snapshot.MarketName
    printfn "⭐ Selection: %s" snapshot.SelectionName
    printfn "💵 Current Price: £%.2f" (float snapshot.CurrentPrice)
    printfn "💰 Total Volume: £%.2f" (float snapshot.TotalVolume)
    printfn "📊 Data Points: %d" snapshot.DataPoints
    
    // Show price trend if we have history
    if marketHistory.Length > 1 then
        let previous = marketHistory.[marketHistory.Length - 2]
        let priceChange = snapshot.CurrentPrice - previous.CurrentPrice
        let volumeChange = snapshot.TotalVolume - previous.TotalVolume
        
        let priceIcon = if priceChange > 0m then "📈" elif priceChange < 0m then "📉" else "➡️"
        let volumeIcon = if volumeChange > 0m then "⬆️" else "⬇️"
        
        printfn "\n📈 TRENDS:"
        printfn "   %s Price Change: £%.3f" priceIcon (float priceChange)
        printfn "   %s Volume Change: £%.2f" volumeIcon (float volumeChange)

let createLiveChart (history: MarketSnapshot list) =
    if history.Length < 2 then
        printfn "📊 Collecting data for charting... (need at least 2 data points)"
        None
    else
        let times = history |> List.map (fun s -> s.Timestamp)
        let prices = history |> List.map (fun s -> float s.CurrentPrice)
        let volumes = history |> List.map (fun s -> float s.TotalVolume)
        
        let priceChart = 
            Chart.Scatter(
                x = times,
                y = prices,
                mode = StyleParam.Mode.Lines_Markers,
                Name = "Live Price"
            )
            |> Chart.withLineStyle(Width = 3., Color = Color.fromHex "#E74C3C")
            |> Chart.withMarkerStyle(Size = 8, Color = Color.fromHex "#E74C3C")
            |> Chart.withTraceInfo("Live Price", YAxis = StyleParam.SubPlotId.YAxis 1)
        
        let volumeChart = 
            Chart.Scatter(
                x = times,
                y = volumes,
                mode = StyleParam.Mode.Lines_Markers,
                Name = "Cumulative Volume"
            )
            |> Chart.withLineStyle(Width = 2., Color = Color.fromHex "#3498DB", Dash = StyleParam.DrawingStyle.Dash)
            |> Chart.withMarkerStyle(Size = 6, Color = Color.fromHex "#3498DB")
            |> Chart.withTraceInfo("Cumulative Volume", YAxis = StyleParam.SubPlotId.YAxis 2)
        
        let chart = 
            [priceChart; volumeChart]
            |> Chart.combine
            |> Chart.withTitle(
                Title.init(
                    Text = "🔴 LIVE Market Monitor",
                    Font = Font.init(Size = 18., Color = Color.fromHex "#E74C3C")
                )
            )
            |> Chart.withXAxisStyle(
                Title.init("Time", Font = Font.init(Size = 12.)),
                ShowGrid = true
            )
            |> Chart.withYAxisStyle(
                Title.init("Price (£)", Font = Font.init(Size = 12., Color = Color.fromHex "#E74C3C")),
                Side = StyleParam.Side.Left,
                Id = StyleParam.SubPlotId.YAxis 1
            )
            |> Chart.withY2AxisStyle(
                Title.init("Volume (£)", Font = Font.init(Size = 12., Color = Color.fromHex "#3498DB")),
                Side = StyleParam.Side.Right,
                Overlaying = StyleParam.LinearAxisId.Y,
                Id = StyleParam.SubPlotId.YAxis 2
            )
            |> Chart.withSize(1000, 500)
        
        Some chart

## Single Market Update

Run this cell to fetch the latest market data and update the dashboard.

In [ ]:
// Fetch latest market data
let! snapshotResult = fetchMarketSnapshot config

match snapshotResult with
| Error msg -> 
    printfn "❌ %s" msg
| Ok snapshot ->
    // Check for alerts
    let previousSnapshot = if marketHistory.IsEmpty then None else Some marketHistory.[marketHistory.Length - 1]
    let newAlerts = checkForAlerts snapshot previousSnapshot
    
    // Update history
    marketHistory <- marketHistory @ [snapshot]
    priceAlerts <- priceAlerts @ newAlerts
    
    // Display dashboard
    displayMarketDashboard snapshot
    
    // Display alerts
    displayAlerts newAlerts
    
    // Show chart if we have enough data
    match createLiveChart marketHistory with
    | None -> ()
    | Some chart -> chart

## Market History Analysis

In [ ]:
// Analyze market history
if marketHistory.IsEmpty then
    printfn "📊 No market history available yet. Run the update cell above first."
else
    printfn "\n📈 MARKET HISTORY ANALYSIS"
    printfn "==========================="
    printfn "📊 Total Snapshots: %d" marketHistory.Length
    printfn "🕐 First Update: %s" (marketHistory.[0].Timestamp.ToString("HH:mm:ss"))
    printfn "🕐 Last Update: %s" (marketHistory.[marketHistory.Length - 1].Timestamp.ToString("HH:mm:ss"))
    
    let prices = marketHistory |> List.map (fun s -> s.CurrentPrice)
    let volumes = marketHistory |> List.map (fun s -> s.TotalVolume)
    
    let minPrice = List.min prices
    let maxPrice = List.max prices
    let avgPrice = List.average prices
    let latestVolume = List.last volumes
    
    printfn "\n💰 PRICE STATISTICS:"
    printfn "   📉 Minimum: £%.3f" (float minPrice)
    printfn "   📈 Maximum: £%.3f" (float maxPrice)
    printfn "   📊 Average: £%.3f" (float avgPrice)
    printfn "   📊 Range: £%.3f" (float (maxPrice - minPrice))
    printfn "   💰 Latest Volume: £%.2f" (float latestVolume)
    
    printfn "\n🚨 ALERT SUMMARY:"
    if priceAlerts.IsEmpty then
        printfn "   ✅ No alerts triggered"
    else
        let volumeAlerts = priceAlerts |> List.filter (fun a -> a.AlertType = "VOLUME")
        let priceAlerts' = priceAlerts |> List.filter (fun a -> a.AlertType = "PRICE_CHANGE")
        printfn "   💰 Volume Alerts: %d" volumeAlerts.Length
        printfn "   📈 Price Change Alerts: %d" priceAlerts'.Length

## Reset Monitoring Data

Run this cell to clear all collected data and start fresh.

In [ ]:
// Reset all monitoring data
marketHistory <- []
priceAlerts <- []

printfn "🔄 Market monitoring data has been reset."
printfn "📊 Run the 'Single Market Update' cell to start collecting new data."